In [1]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

c:\Users\PhnaeendraAdmin\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


c:\Users\PhnaeendraAdmin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PhnaeendraAdmin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


c:\Users\PhnaeendraAdmin\anaconda3\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map

c:\Users\PhnaeendraAdmin\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [3]:
import pandas as pd
from transformers import AutoTokenizer, pipeline

# Load pre-trained tokenizer and classifier (using your specific classifier)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Load the CSV files
lyrics_df = pd.read_csv("lyrics_CSV.csv")
lyrics_sum_df = pd.read_csv("Sum_lyrics.csv")

# Ensure both files have the same number of rows
if len(lyrics_df) != len(lyrics_sum_df):
    raise ValueError("The two CSV files must have the same number of rows.")

# Function to truncate lyrics to 512 tokens
def truncate_to_512_tokens(text):
    encoded_input = tokenizer(text, truncation=True, max_length=512, return_tensors="pt")
    return tokenizer.decode(encoded_input['input_ids'][0], skip_special_tokens=True)

# Function to classify lyrics and return the emotion with the highest score
def get_top_emotion(text):
    result = classifier(text)[0]
    # Select the label with the highest score from the returned scores
    return max(result, key=lambda x: x['score'])['label']

# Add an Emotion column to both DataFrames
lyrics_df['Emotion'] = lyrics_df['Lyrics'].apply(lambda x: get_top_emotion(truncate_to_512_tokens(x)))
lyrics_sum_df['Emotion'] = lyrics_sum_df['Sum Lyrics'].apply(lambda x: get_top_emotion(x))

# Now compare the Emotion columns line by line
total_songs = len(lyrics_df)
matches = sum(lyrics_df['Emotion'].str.lower() == lyrics_sum_df['Emotion'].str.lower())

# Calculate the accuracy percentage
accuracy = (matches / total_songs) * 100

# Print the accuracy result
print(f"Accuracy of emotion matching between truncated and non-truncated lyrics: {accuracy:.2f}%")

# Save the updated CSVs with Emotion fields
lyrics_df.to_csv("lyrics_with_emotions.csv", index=False)
lyrics_sum_df.to_csv("lyrics_sum_with_emotions.csv", index=False)


Accuracy of emotion matching between truncated and non-truncated lyrics: 79.61%


In [4]:
import pandas as pd
from transformers import AutoTokenizer, pipeline

# Load pre-trained tokenizer and classifier
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Load the CSV files
lyrics_df = pd.read_csv("lyrics_CSV.csv")
lyrics_sum_df = pd.read_csv("Sum_lyrics.csv")

# Function to split lyrics into chunks of less than 512 tokens
def split_into_chunks(text, max_tokens=512):
    tokens = tokenizer.encode(text)
    return [tokenizer.decode(tokens[i:i + max_tokens], skip_special_tokens=True) 
             for i in range(0, len(tokens), max_tokens) if len(tokens[i:i + max_tokens]) < max_tokens]

# Function to classify and average scores
def classify_and_average(text):
    chunks = split_into_chunks(text)
    total_scores = {}
    num_chunks = len(chunks)

    for chunk in chunks:
        results = classifier(chunk)[0]
        for result in results:
            label = result['label']
            score = result['score']
            if label not in total_scores:
                total_scores[label] = 0
            total_scores[label] += score
    
    # Average scores
    for label in total_scores:
        total_scores[label] /= num_chunks
    
    # Get the emotion with the highest average score
    return max(total_scores, key=total_scores.get)

# Add an Emotion column to the lyrics DataFrame
lyrics_df['Emotion'] = lyrics_df['Lyrics'].apply(classify_and_average)

# For the second file, classify the full lyrics
lyrics_sum_df['Emotion'] = lyrics_sum_df['Sum Lyrics'].apply(lambda x: get_top_emotion(x))

# Compare the Emotion columns line by line
total_songs = len(lyrics_df)
matches = sum(lyrics_df['Emotion'].str.lower() == lyrics_sum_df['Emotion'].str.lower())

# Calculate the accuracy percentage
accuracy = (matches / total_songs) * 100

# Print the accuracy result
print(f"Accuracy of emotion matching between split and non-split lyrics: {accuracy:.2f}%")

# Save the updated CSVs with Emotion fields
lyrics_df.to_csv("lyrics_with_emotions.csv", index=False)
lyrics_sum_df.to_csv("lyrics_sum_with_emotions.csv", index=False)


c:\Users\PhnaeendraAdmin\anaconda3\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map

Accuracy of emotion matching between split and non-split lyrics: 76.34%
